<a href="https://colab.research.google.com/github/RadhavaramHarika/Harika_INFO5731_Spring2020/blob/master/Harika_In_class_exercise_04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **The fourth in-class-exercise**

# 1. Text Data Preprocessing

Here is a [legal case](https://github.com/unt-iialab/INFO5731_Spring2020/blob/master/In_class_exercise/01-05-1%20%20Adams%20v%20Tanner.txt) we collected from westlaw, please follow the steps we mentioned in lesson 5 to clean the data:



## 1.1 Basic feature extraction using text data

*   Number of sentences
*   Number of words
*   Number of characters
*   Average word length
*   Number of stopwords
*   Number of special characters
*   Number of numerics
*   Number of uppercase words

## 1.2 Basic Text Pre-processing of text data

*   Lower casing
*   Punctuation removal
*   Stopwords removal
*   Frequent words removal
*   Rare words removal
*   Spelling correction
*   Tokenization
*   Stemming
*   Lemmatization

## 1.3 Save all the **clean sentences** to a **csv file** (one column, each raw is a sentence) after finishing all the steps above.


## 1.4 Advance Text Processing

*   Calculate the term frequency of all the terms.
*   Print out top 10 1-gram, top 10 2-grams, and top 10 2-grams terms as features.


In [3]:
# Write your code here
import nltk
import csv,os,math,unicodedata,re
from google.colab import drive,files
nltk.download('punkt')
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
from nltk.corpus import stopwords
from collections import Counter 
from textblob import TextBlob
from nltk.stem import PorterStemmer
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
import pandas as panda
from sklearn.feature_extraction.text import TfidfVectorizer

#Using functions to open text file from gdrive and do basic fetaure extraction and basic preprocessing
def open_textfile():
  drive.mount('/content/gdrive')
  file_list = os.listdir('/content/gdrive/My Drive/Colab Notebooks')
  os.chdir('/content/gdrive/My Drive/Colab Notebooks')
  if 'Legal_case.txt' in file_list:
    text_data = open('Legal_case.txt').read()
  clean_data = text_data.replace('\xa0','')
  return clean_data

def getTextSentences(samp_data):
  text_sent = [each for each in samp_data.split('\n')
                  if each]
  return text_sent

def basic_featExtraction(file_data):
  data_dict = {}
  text_words = file_data.split()
  text_sent = [each for each in file_data.split('\n')
                  if each]
  print("Number of sentences: ",len(text_sent))
  stop_words = stopwords.words("english")

  for each_sent in text_sent:
    count = stop_count = spl_c = num_c = up_c = 0
    words = each_sent.split()
    for each in word_tokenize(each_sent):
      if each.isdigit():
        num_c += 1
    for each_word in words:
      spl_w = re.sub(r"\w","", each_word, flags = re.I)
      if spl_w:
        spl_c += len(spl_w)
      count += len(each_word)
      if each_word.lower() in stop_words:
        stop_count += 1
      if each_word.isupper():
        up_c += 1
    data_dict[each_sent] = [len(words),count,count/len(words), stop_count, spl_c, num_c,up_c]
    return data_dict

def uploadBasicExtractFeatToCSV(samp_dict):
  datafrm = panda.DataFrame({"Sentences": samp_dict.keys(),
                            "No. of words": [samp_dict[k][0] for k,v in samp_dict.items()],
                            "No. of Characters without spaces":[samp_dict[k][1] for k,v in samp_dict.items()],
                            "Avg Word Length":[samp_dict[k][2] for k,v in samp_dict.items()],
                            "No. of stopwords":[samp_dict[k][3] for k,v in samp_dict.items()],
                            "No. of special chars":[samp_dict[k][4] for k,v in samp_dict.items()],
                            "No. of numerics":[samp_dict[k][5] for k,v in samp_dict.items()],
                            "No. of uppercase words":[samp_dict[k][6] for k,v in samp_dict.items()]})

  datafrm.to_csv("Clean.csv")
  return datafrm

def freq_words(samp_data):
  split_it = samp_data.split() 
  counter = Counter(split_it)
  most_occur = {}
  for each in counter.most_common(10):
    most_occur[each[0]] = each[1]
  return most_occur

def rare_words(samp_data):
  split_it = samp_data.split() 
  counter = Counter(split_it)
  rare_occur = {}
  for each in counter.most_common()[-10:]:
    rare_occur[each[0]] = each[1]
  return rare_occur

def lower_sentences(samp_sent):
  lowers = []
  for each in samp_sent:
    lowers.append(each.lower())
  return lowers

def remove_punctuations(samp_sent):
  removed_punct = []
  for each in samp_sent:
    removed_punct.append(re.sub(r"\W", " ", each, flags=re.I))
  return removed_punct

def remove_stop_words(samp_sent):
  stops = stopwords.words('english')
  no_stops = []
  for each in samp_sent:
    words = each.split()
    no_stops.append(' '.join([w for w in words
                              if w not in stops]))
  return no_stops

def remove_freq(samp_sent):
  freq = freq_words(' '.join(samp_sent)).keys()
  remove_freq = [] 
  for each in samp_sent:
    words = each.split()
    remove_freq.append(' '.join([w for w in words
                              if w not in freq]))
  return remove_freq

def remove_rare(samp_sent):
  rare = rare_words(' '.join(samp_sent)).keys()
  remove_rare = [] 
  for each in samp_sent:
    words = each.split()
    remove_rare.append(' '.join([w for w in words
                              if w not in rare]))
  return remove_rare

def check_correct_spelling(samp_sent):
  corrected_sent = []
  for each in samp_sent:
    words = each.split()
    corrected_sent.append(' '.join([str(TextBlob(w).correct()) for w in words]))
  return corrected_sent

def tokenized(samp_sent):
  tokens = []
  for each in samp_sent:
    tokens.append(word_tokenize(each))
  return tokens

def stemming(samp_sent_tokens):
  stem_sent = []
  ps = PorterStemmer()
  for each in samp_sent_tokens:
    stem_sent.append([ps.stem(w) for w in each])
  return stem_sent

def lemmatized_sent_tokens(samp_sent_tokens):
  lmtzr = WordNetLemmatizer() 
  lem_sent_tokens = []
  for each in samp_sent_tokens:
    lem_sent_tokens.append([lmtzr.lemmatize(w) for w in each])
  return lem_sent_tokens

def uploadPreprocessedDataToCSV(clean_data,samp_sent):
  global final_tokens
  lowered_sent = lower_sentences(text_sentences)
  no_punct_sent = remove_punctuations(lowered_sent)
  no_stop_sent = remove_stop_words(no_punct_sent)
  no_freq_sent = remove_freq(no_stop_sent)
  no_rare_sent = remove_rare(no_freq_sent)
  corrected_sent = check_correct_spelling(no_rare_sent)
  token_sent = tokenized(corrected_sent)
  stem_sent_tokens = stemming(token_sent)
  lemmatized_tokens = lemmatized_sent_tokens(stem_sent_tokens)
  final_tokens = lemmatized_tokens
  csv_dict = {"Sentences":samp_sent,
              "LowerCased":lowered_sent,
              "PunctuationRemoved":no_punct_sent,
              "StopWordsRemoved":no_stop_sent,
              "FreqWordsRemoved":no_freq_sent,
              "RareWordsRemoved":no_rare_sent,
              "CorrectedSpelling":corrected_sent,
              "Tokenized":token_sent,
              "Stemming":stem_sent_tokens,
              "Lemmatized":lemmatized_tokens}

  datafr = panda.DataFrame(csv_dict)
  return datafr

#1.1 Calling above functions to do basic feature extraction 
cleaned_data = open_textfile()
text_sentences = getTextSentences(cleaned_data)
csv_dict = basic_featExtraction(cleaned_data)
print(uploadBasicExtractFeatToCSV(csv_dict))

#1.2 Calling above functions to do basic pre processing 
final_tokens = []
uploadPreprocessedDataToCSV(cleaned_data,text_sentences)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
Number of sentences:  147
      Sentences  No. of words  ...  No. of numerics  No. of uppercase words
0  (5 Ala. 740)             3  ...                2                       0

[1 rows x 8 columns]


,Sentences,LowerCased,PunctuationRemoved,StopWordsRemoved,FreqWordsRemoved,RareWordsRemoved,CorrectedSpelling,Tokenized,Stemming,Lemmatized
0,5 Ala. 740,5 ala. 740,5 ala 740,5 ala 740,5 ala 740,5 ala 740,5 all 740,"[5, all, 740]","[5, all, 740]","[5, all, 740]"
1,Supreme Court of Alabama.,supreme court of alabama.,supreme court of alabama,supreme court alabama,supreme alabama,supreme alabama,supreme alabama,"[supreme, alabama]","[suprem, alabama]","[suprem, alabama]"
2,ADAMS,adams,adams,adams,adams,adams,adams,[adams],[adam],[adam]
3,v.,v.,v,v,,,,[],[],[]
4,TANNER AND HORTON.,tanner and horton.,tanner and horton,tanner horton,tanner horton,tanner horton,manner norton,"[manner, norton]","[manner, norton]","[manner, norton]"
...,...,...,...,...,...,...,...,...,...,...
142,There are no Filings for this citation.,there are no filings for this citation.,there are no filings for this citation,filings citation,filings citation,filings citation,filing situation,"[filing, situation]","[file, situat]","[file, situat]"
143,Negative Treatment,negative treatment,negative treatment,negative treatment,negative treatment,negative treatment,negative treatment,"[negative, treatment]","[neg, treatment]","[neg, treatment]"
144,There are no Negative Treatment results for th...,there are no negative treatment results for th...,there are no negative treatment results for th...,negative treatment results citation,negative treatment results citation,negative treatment results citation,negative treatment results situation,"[negative, treatment, results, situation]","[neg, treatment, result, situat]","[neg, treatment, result, situat]"
145,History,history,history,history,history,history,history,[history],[histori],[histori]


In [5]:
from nltk.util import ngrams
from collections import Counter

#Using functions to calc term frequency matrix and top 10 ngrams
def calc_TF_IDF(vocab,corpus):
  tfidf = TfidfVectorizer(vocabulary = list(set(vocab)), max_features=1000, lowercase=True, analyzer='word',stop_words= 'english', ngram_range = (1,3))
  tfs = tfidf.fit_transform(corpus.values())
  feature_names = tfidf.get_feature_names()
  corpus_index = [n for n in corpus]
  datafr = panda.DataFrame(tfs.T.todense(), index=feature_names, columns=corpus_index)
  return datafr

def bagOfWords(vocab,count):
  ngram_words = Counter(ngrams(vocab, count))
  print("\n"+str(count)+" gram words are :\n",ngram_words.most_common(10))
  return


total_terms = []
for each in final_tokens:
  for w in each:
    total_terms.append(w)
print("Total Vocabulary: \n",total_terms)
corpus = {}
for i in range(len(text_sentences)):
  corpus[i+1] = text_sentences[i]

#1.3 Calling above functions to print results
bagOfWords(total_terms,1)
bagOfWords(total_terms,2)
bagOfWords(total_terms,3)

calc_TF_IDF(total_terms,corpus)



Total Vocabulary: 
 ['5', 'all', '740', 'suprem', 'alabama', 'adam', 'manner', 'norton', 'june', 'term', '1843', 'sycosi', 'writ', 'error', 'circuit', 'sumter', 'west', 'headnot', '2', '1', 'chattel', 'mortgag', 'crop', 'grow', 'exist', 'subject', 'matter', 'mortgag', 'contract', 'pas', 'interest', 'vest', 'posse', 'either', 'immedi', 'futur', 'time', '4', 'case', 'cite', 'headnot', '2', 'creditor', 'remedi', 'prioriti', 'st', '1821', 'prohibit', 'attach', 'favor', 'i', 'a', 'grow', 'attach', '5', 'case', 'cite', 'headnot', '1', 'trial', 'properti', 'statut', 'novemb', '1840', 'issu', 'circuit', 'sumter', 'suit', 'plaintiff', 'error', 'requir', 'sheriff', 'counti', 'make', 'good', 'c', 'allen', 'harrison', 'other', 'sum', 'thirti', 'seven', 'hundr', 'seventi', 'seven', '80', '100', 'dollar', 'besid', 'cost', 'levi', 'thirti', 'bale', 'cotton', 'properti', 'allen', 'harrison', 'claim', 'bond', 'given', 'tri', 'issu', 'made', 'tri', 'question', 'liabil', 'cotton', 'plaintiff', 'submit', 

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147
porter,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.183672,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
protect,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
thomson,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
21,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
suspend,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.069497,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
oblig,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
assign,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.179509,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
though,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
construct,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# 2. Python Regular Expression

## 2.1 Write a Python program to remove leading zeros from an IP address. 

ip = "260.08.094.109"

In [0]:
# Write your code here
import re

ip = "260.08.094.109"
ip_no_zeros = []
for each in ip.split('.'):
  ip_no_zeros.append(re.sub(r"^0+", "", each))
  
print("\nGiven input IP address :",ip)  
print("\nFinal IP address after removing leading zeros is :",'.'.join(ip_no_zeros))




Given input IP address : 260.08.094.109

Final IP address after removing leading zeros is : 260.8.94.109


## 2.2 Write a Python Program to extract all the years from the following sentence.

sentence = "The 2010s were a dramatic decade, filled with ups and downs, more than 1000 stroies have happened. As the decade comes to a close, Insider took a look back at some of the biggest headline-grabbing stories, from 2010 to 2019. The result was 119 news stories that ranged from the heartwarming rescue of a Thai boys' soccer team from a flooded cave to the divisive election of President Donald Trump."

In [0]:
# Write your code here
nltk.download('punkt')
from nltk.tokenize import word_tokenize
import re

def extract_years(samp_sentence):
  num_sent = re.sub(r"[a-zA-Z]", "", samp_sentence, flags=re.I)
  sentn_list = word_tokenize(num_sent)
  num_list = [int(each) for each in sentn_list 
              if each.isnumeric() and int(each)>1500]
  
  return num_list

sentence = "The 2010s were a dramatic decade, filled with ups and downs, more than 1000 stroies have \
happened. As the decade comes to a close, Insider took a look back at some of the biggest headline-grabbing \
stories, from 2010 to 2019. The result was 119 news stories that ranged from the heartwarming rescue of a Thai boys' \
soccer team from a flooded cave to the divisive election of President Donald Trump."  
print("\nGiven sentence is : \n",sentence)
print("\nResultant years from above sentence are :\n",extract_years(sentence))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!

Given sentence is : 
 The 2010s were a dramatic decade, filled with ups and downs, more than 1000 stroies have happened. As the decade comes to a close, Insider took a look back at some of the biggest headline-grabbing stories, from 2010 to 2019. The result was 119 news stories that ranged from the heartwarming rescue of a Thai boys' soccer team from a flooded cave to the divisive election of President Donald Trump.

Resultant years from above sentence are :
 [2010, 2010, 2019]
